In [2]:
%pip install datasets
%pip install scikit-learn


Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Users\ekagr\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


You should consider upgrading via the 'c:\Users\ekagr\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


In [3]:
import pandas as pd


# Load the dataset
dataset = pd.read_csv('twitter_training.csv')
print(dataset.head())


   2401  Borderlands  Positive  \
0  2401  Borderlands  Positive   
1  2401  Borderlands  Positive   
2  2401  Borderlands  Positive   
3  2401  Borderlands  Positive   
4  2401  Borderlands  Positive   

  im getting on borderlands and i will murder you all ,  
0  I am coming to the borders and I will kill you...     
1  im getting on borderlands and i will kill you ...     
2  im coming on borderlands and i will murder you...     
3  im getting on borderlands 2 and i will murder ...     
4  im getting into borderlands and i can murder y...     


In [4]:
dataset.columns = ['ID', 'Feature', 'Sentiment', 'Text']
print(dataset.head())


     ID      Feature Sentiment  \
0  2401  Borderlands  Positive   
1  2401  Borderlands  Positive   
2  2401  Borderlands  Positive   
3  2401  Borderlands  Positive   
4  2401  Borderlands  Positive   

                                                Text  
0  I am coming to the borders and I will kill you...  
1  im getting on borderlands and i will kill you ...  
2  im coming on borderlands and i will murder you...  
3  im getting on borderlands 2 and i will murder ...  
4  im getting into borderlands and i can murder y...  


In [5]:
dataset = dataset[['Text', 'Sentiment']]
dataset.head(10)

,Text,Sentiment
0,I am coming to the borders and I will kill you...,Positive
1,im getting on borderlands and i will kill you ...,Positive
2,im coming on borderlands and i will murder you...,Positive
3,im getting on borderlands 2 and i will murder ...,Positive
4,im getting into borderlands and i can murder y...,Positive
5,So I spent a few hours making something for fu...,Positive
6,So I spent a couple of hours doing something f...,Positive
7,So I spent a few hours doing something for fun...,Positive
8,So I spent a few hours making something for fu...,Positive
9,2010 So I spent a few hours making something f...,Positive


In [6]:
dataset = dataset.dropna(subset=['Text', 'Sentiment'])


In [7]:
print(dataset['Sentiment'].value_counts())


Sentiment
Negative      22358
Positive      20654
Neutral       18108
Irrelevant    12875
Name: count, dtype: int64


In [8]:
label_mapping = {'Positive': 2, 'Neutral': 1, 'Negative': 0, 'Irrelevant': 3}
dataset['Sentiment'] = dataset['Sentiment'].map(label_mapping)


In [9]:
# 2 - Positive
# 1 - Neutral
# 0 - Negative
# 3 - Irrelevant

print(dataset['Sentiment'].value_counts())

Sentiment
0    22358
2    20654
1    18108
3    12875
Name: count, dtype: int64


In [10]:
import re

def clean_text(text):
    text = re.sub(r"http\S+", "", text)  # Remove URLs
    text = re.sub(r"@\w+", "", text)    # Remove mentions
    text = re.sub(r"#\w+", "", text)    # Remove hashtags
    text = re.sub(r"\d+", "", text)     # Remove numbers
    text = re.sub(r"[^\w\s]", "", text) # Remove punctuation
    text = text.lower().strip()         # Convert to lowercase and strip whitespace
    return text

dataset['Text'] = dataset['Text'].apply(clean_text)


In [11]:
from transformers import DistilBertTokenizer

tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
tokenized_data = tokenizer(list(dataset['Text']), padding=True, truncation=True, max_length=128, return_tensors="pt")

# Convert labels to tensor
import torch
labels = torch.tensor(dataset['Sentiment'].tolist())


Splitting Dataset


In [12]:
from sklearn.model_selection import train_test_split

train_texts, test_texts, train_labels, test_labels = train_test_split(
    tokenized_data['input_ids'], labels, test_size=0.2, random_state=42
)


In [13]:
from transformers import DistilBertForSequenceClassification, Trainer, TrainingArguments
from datasets import Dataset
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, f1_score, precision_score, recall_score
import random
import numpy as np


def compute_metrics(eval_pred):

    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return {
        'accuracy': accuracy_score(labels, predictions),
        'f1': f1_score(labels, predictions, average='weighted'),
        'precision': precision_score(labels, predictions, average='weighted'),
        'recall': recall_score(labels, predictions, average='weighted')
    }

model = DistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=4)

# Convert the datasets to the appropriate format
train_dataset = Dataset.from_dict({"input_ids": train_texts, "labels": train_labels})
eval_dataset = Dataset.from_dict({"input_ids": test_texts, "labels": test_labels})


eval_dataset_list = list(eval_dataset)  # Convert to a list
sampled_eval_dataset = random.sample(eval_dataset_list, 4500)  # Take 4500 random samples

# Convert back to a Dataset object
eval_dataset = Dataset.from_list(sampled_eval_dataset)


training_args = TrainingArguments(
    output_dir='./results',  # Directory to save model and checkpoints
    evaluation_strategy="steps",  # Evaluate only at the end of each epoch
    save_strategy="steps",  # Save checkpoints regularly
    save_total_limit=3,  # Number of maximum checkpoints to save
    save_steps=500,  # Save checkpoints every 500 steps
    eval_steps=500,  # Evaluate model every 500 steps
    logging_dir='./logs',  # Directory for logs
    logging_steps=100,  # Log every 25 steps
    learning_rate=2e-5,  # Use a smaller learning rate for fine-tuning
    per_device_train_batch_size=8,  # Reduce batch size
    per_device_eval_batch_size=8,  # Reduce batch size for evaluation
    num_train_epochs=3,  # Train for a smaller number of epochs
    weight_decay=0.01,  # Regularization to avoid overfitting
    gradient_accumulation_steps=2,  # Accumulate gradients over 2 steps
    no_cuda=True,  # Do not use CUDA (since you’re on a CPU)
    fp16=False,  # Disable mixed precision training
    load_best_model_at_end=True,  # Save the best model at the end of training
    metric_for_best_model="eval_loss",  # Use evaluation loss to track the best model
    greater_is_better=False,  # We want to minimize the loss
    push_to_hub=False,  # If you don't plan to push to the HuggingFace hub
    report_to=["tensorboard"], 
    log_level="info",  
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    compute_metrics=compute_metrics
    
)


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
c:\Users\ekagr\AppData\Local\Programs\Python\Python310\lib\site-packages\transformers\training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
c:\Users\ekagr\AppData\Local\Programs\Python\Python310\lib\site-packages\transformers\training_args.py:1590: FutureWarning: using `no_cuda` is deprecated and will be removed in version 5.0 of 🤗 Transformers. Use `use_cpu` instead
  warnings.warn(


In [14]:
import os
from transformers import Trainer
import numpy as np
from torch.utils.tensorboard import SummaryWriter

# Assuming `training_args` has been defined and training has started before
last_checkpoint = None
checkpoints = [d for d in os.listdir(training_args.output_dir) if 'checkpoint' in d]
if checkpoints:
    last_checkpoint = os.path.join(training_args.output_dir, sorted(checkpoints, key=lambda x: int(x.split('-')[-1]))[-1])


# # sess.graph contains the graph definition; that enables the Graph Visualizer.
# with SummaryWriter(log_dir=training_args.logging_dir) as sw:
#     trainer.train(resume_from_checkpoint=last_checkpoint if last_checkpoint else None)

# Now, resume from the last checkpoint
trainer.train(resume_from_checkpoint=last_checkpoint if last_checkpoint else None)



# Load the TensorBoard notebook extension

writer = SummaryWriter(log_dir=training_args.logging_dir)

# Example: Log a custom scalar
writer.add_scalar("custom_metric", 0.95, global_step=1)

# Close the writer after training
writer.close()

# TensorBoard log directory setting
{
  "type": "setting",
  "settings": {
    "python.tensorboard.logDirectory": "./logs"
  }
}


Loading model from ./results\checkpoint-8400.
c:\Users\ekagr\AppData\Local\Programs\Python\Python310\lib\site-packages\transformers\trainer.py:3418: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this ex

{'loss': 0.2737, 'grad_norm': 13.347362518310547, 'learning_rate': 4.684684684684685e-06, 'epoch': 2.3}


                                                      
 77%|███████▋  | 8500/11100 [14:38<3:14:57,  4.50s/it]

{'eval_loss': 0.47807833552360535, 'eval_accuracy': 0.8448888888888889, 'eval_f1': 0.8441686819933549, 'eval_precision': 0.8450396337253958, 'eval_recall': 0.8448888888888889, 'eval_runtime': 355.6528, 'eval_samples_per_second': 12.653, 'eval_steps_per_second': 1.583, 'epoch': 2.3}


 77%|███████▋  | 8600/11100 [22:24<3:16:37,  4.72s/it]  Saving model checkpoint to ./results\checkpoint-8600
Configuration saved in ./results\checkpoint-8600\config.json


...

Model weights saved in ./results\checkpoint-11000\model.safetensors
Deleting older checkpoint [results\checkpoint-10400] due to args.save_total_limit
100%|██████████| 11100/11100 [4:22:07<00:00,  3.81s/it]   Saving model checkpoint to ./results\checkpoint-11100
Configuration saved in ./results\checkpoint-11100\config.json


{'loss': 0.2995, 'grad_norm': 35.4686164855957, 'learning_rate': 0.0, 'epoch': 3.0}


Model weights saved in ./results\checkpoint-11100\model.safetensors
Deleting older checkpoint [results\checkpoint-10600] due to args.save_total_limit


Training completed. Do not forget to share your model on huggingface.co/models =)


Loading best model from ./results\checkpoint-11000 (score: 0.43545860052108765).
100%|██████████| 11100/11100 [4:22:10<00:00,  1.42s/it]


{'train_runtime': 15730.4082, 'train_samples_per_second': 11.289, 'train_steps_per_second': 0.706, 'train_loss': 0.06736033242028039, 'epoch': 3.0}


{'type': 'setting', 'settings': {'python.tensorboard.logDirectory': './logs'}}

In [15]:

finaleval = Dataset.from_list(eval_dataset_list) 


trainer.evaluate(eval_dataset=finaleval)



***** Running Evaluation *****
  Num examples = 14799
  Batch size = 8
100%|██████████| 1850/1850 [19:05<00:00,  1.61it/s]


{'eval_loss': 0.4403340518474579,
 'eval_accuracy': 0.8553280627069396,
 'eval_f1': 0.8551453091158301,
 'eval_precision': 0.8557576722681051,
 'eval_recall': 0.8553280627069396,
 'eval_runtime': 1146.4615,
 'eval_samples_per_second': 12.908,
 'eval_steps_per_second': 1.614,
 'epoch': 3.0}

In [16]:
model.save_pretrained("./sentiment_model")
tokenizer.save_pretrained("./sentiment_model")


Configuration saved in ./sentiment_model\config.json
Model weights saved in ./sentiment_model\model.safetensors
tokenizer config file saved in ./sentiment_model\tokenizer_config.json
Special tokens file saved in ./sentiment_model\special_tokens_map.json


('./sentiment_model\\tokenizer_config.json',
 './sentiment_model\\special_tokens_map.json',
 './sentiment_model\\vocab.txt',
 './sentiment_model\\added_tokens.json')

In [17]:
from IPython.display import Javascript

# Save the current notebook
def save_current_notebook():
    print("Saving the current notebook...")
    display(Javascript('IPython.notebook.save_checkpoint();'))

save_current_notebook()


Saving the current notebook...


<IPython.core.display.Javascript object>

In [18]:
# WARNING WILL SHUTDOWN YOUR COMPUTER

# import ctypes
# user32 = ctypes.WinDLL('user32')
# user32.ExitWindowsEx(0x00000008, 0x00000000)

0